# Example 2. Reconstruction parameters: Resolution target 2D-SIM
  
Test targets with known ground truth and a variety of pattern orientations are useful to estimate the overall quality of the SIM instrument and reconstruction. Here, we use an [Argolight SIM v1 slide](https://argolight.com/files/Argo-SIM/Argo-SIM-v1.1_User-guide.pdf) with sets of lines that are spaced from 0 to 390 nm apart. The line spacing changes in 30 nm increments.

### Import libraries.

In [ ]:
import napari
import numpy as np
from numpy import fft
import json
import tifffile
from pathlib import Path
import mcsim.analysis.sim_reconstruction as sim
import localize_psf.fit_psf as psf
import localize_psf.affine as affine
import localize_psf.rois as rois
from mcsim.analysis import analysis_tools
import matplotlib.pyplot as plt
from matplotlib.colors import PowerNorm

### Load experimental data containing all wavelengths and patterns.

In [ ]:
ncolors = 2 # number of channels
nangles = 3 # number of angles
nphases = 3 # number of phases
nz = 1 # number of z planes
nx = 2048 # number of x pixels on camera
ny = 2048 # number of y pixels on camera

# read data from disk
imgs = tifffile.imread(Path("data", "example_002", "raw_data", "argosim_line_pairs.tif")).reshape([ncolors, nangles, nphases, ny, nx])

### Set experimental metadata.

In [ ]:
na = 1.3
pixel_size = 0.065 # um
excitation_wavelengths = [0.465, 0.532] # um
emission_wavelengths = [0.519, 0.580] # um

### View data in Napari

In [ ]:
# define colormaps
colormaps = ['bop blue', 'bop orange']

# add images to napari viewer with scale information, colormaps, and additive blending
viewer = napari.view_image(imgs[0,:],name='em: 520 nm',scale=(1,pixel_size,pixel_size),colormap = colormaps[0],blending='additive',contrast_limits=[0,2**16-1])
viewer.add_image(imgs[1,:],name='em: 580 nm',scale=(1,pixel_size,pixel_size),colormap = colormaps[1],blending='additive',contrast_limits=[0,2**16-1])

# activate scale bar in physical units
viewer.scale_bar.unit='um'
viewer.scale_bar.visible=True

# label sliders in napari viewer
viewer.dims.axis_labels = ['a','p','y','x']

### Crop image to ROI containing Argolight test target.

In [ ]:
# [cy, cx]
roi = rois.get_centered_roi([791, 896], [850, 850])
nx_roi = roi[3] - roi[2]
ny_roi = roi[1] - roi[0]

## SIM reconstruction using calibrated instrument parameters 

### Load experimental optical transfer function.

In [ ]:
otf_data_path = Path("data", "example_002", "calibration", "2020_05_19_otf_fit_blue.json")

# load optical transfer function data
with open(otf_data_path, 'rb') as f:
    otf_data = json.load(f)
otf_p = np.asarray(otf_data['fit_params'])

# define function to return optical transfer function
otf_fn = lambda f, fmax: 1 / (1 + (f / fmax * otf_p[0]) ** 2) * psf.circ_aperture_otf(f, 0, na, 2 * na / fmax)

### Load pre-calculated affine transforms for qi2lab DMD-SIM.

In [ ]:
affine_fnames = [Path("data", "example_002", "calibration", "2021-02-03_09;43;06_affine_xform_blue_z=0.json"),
                 Path("data", "example_002", "calibration", "2021-02-03_09;43;06_affine_xform_green_z=0.json")]

# load affine transforms for each channel
affine_xforms = []
for p in affine_fnames:
    with open(p, 'rb') as f:
        affine_xforms.append(np.asarray(json.load(f)['affine_xform']))

### Load qi2lab DMD-SIM patterns. Estimate frequency and phase using extracted affine transforms.

In [ ]:
dmd_pattern_data_fpath = [Path("data", "example_002", "calibration", "sim_patterns_period=6.01_nangles=3.json"),
                          Path("data", "example_002", "calibration", "sim_patterns_period=6.82_nangles=3.json")]

# variables to store information on frequences and phases of DMD patterns
frqs_dmd = np.zeros((2, 3, 2))
phases_dmd = np.zeros((ncolors, nangles, nphases))

# loop over all channels
for kk in range(ncolors):

    # load patterns displayed on DMD for current channel
    ppath = dmd_pattern_data_fpath[kk]
    with open(ppath, 'rb') as f:
        pattern_data = json.load(f)

    # load affine transform for current channel
    xform = affine_xforms[kk]

    # DMD intensity frequency and phase (twice electric field frq/phase)
    frqs_dmd[kk] = 2 * np.asarray(pattern_data['frqs'])
    phases_dmd[kk] = 2 * np.asarray(pattern_data['phases'])
    dmd_nx = int(pattern_data['nx'])
    dmd_ny = int(pattern_data['ny'])

# loop over all channels
# here, we limit to first channel because we will only reconstruct the first channel
ncolors_limited = 1
for kk in range(ncolors_limited):
    
    # calculate optical transfer function matrix
    fmax = 1 / (0.5 * emission_wavelengths[kk] / na)
    fx = fft.fftshift(fft.fftfreq(nx_roi, pixel_size))
    fy = fft.fftshift(fft.fftfreq(ny_roi, pixel_size))
    ff = np.sqrt(fx[None, :] ** 2 + fy[:, None] ** 2)
    otf = otf_fn(ff, fmax)
    otf[ff >= fmax] = 0

    # guess frequencies/phases using OTF, affine transform, and DMD patterns
    frqs_guess = np.zeros((nangles, 2))
    phases_guess = np.zeros((nangles, nphases))
    for ii in range(nangles):
        for jj in range(nphases):
            # estimate frequencies based on affine_xform
            frqs_guess[ii, 0], frqs_guess[ii, 1], phases_guess[ii, jj] = \
                affine.xform_sinusoid_params_roi(frqs_dmd[kk, ii, 0], frqs_dmd[kk, ii, 1],
                                                 phases_dmd[kk, ii, jj], [dmd_ny, dmd_nx], roi, xform)

    # convert from 1/mirrors to 1/um
    frqs_guess = frqs_guess / pixel_size

#### Expand to see detailed information on SIM reconstruction class



Reconstruct raw SIM data into widefield, SIM-SR, SIM-OS, and deconvolved images using the Wiener filter style reconstruction of Gustafsson and Heintzmann. This code relies on various ideas developed and implemented elsewhere, see for example fairSIM and openSIM.

An instance of this class may be used directly to reconstruct a single SIM image which is stored as a numpy array. For a typical experiment it is usually best to write a helper function to load the data and coordinate the SIM parameter estimation and reconstruction of e.g. various channels, z-slices, time points or etc. For an example of this approach, see the function reconstruct_mm_sim_dataset()

*:param physical_params:* {'pixel_size', 'na', 'wavelength'}. Pixel size and emission wavelength in um

*:param imgs:* nangles x nphases x ny x nx raw data to be reconstructed

*:param otf:* optical transfer function evaluated at the same frequencies as the fourier transforms of imgs. If None, estimate from NA. This can either be an array of size ny x nx, or an array of size nangles x ny x nx The second case corresponds to a system that has different OTF's per SIM acquisition angle.

*:param wiener_parameter:* Attenuation parameter for Wiener filtering. This has a sligtly different meaning depending on the value of combine_bands_mode

*:param str frq_estimation_mode:* "band-correlation", "fourier-transform", or "fixed" "band-correlation" first unmixes the bands using the phase guess values and computes the correlation between the shifted and unshifted band "fourier-transform" correlates the Fourier transform of the image with itself. "fixed" uses the frq_guess values

*:param frq_guess:* 2 x nangles array of guess SIM frequency values

*:param str phase_estimation_mode:* "wicker-iterative", "real-space", "naive", or "fixed"
"wicker-iterative" follows the approach of https://doi.org/10.1364/OE.21.002032.\ "real-space" follows the approach of section IV-B in https://doir.org/10.1109/JSTQE.2016.2521542. "naive" uses the phase of the Fourier transform of the raw data. "fixed" uses the values provided from phases_guess.

*:param phases_guess:* nangles x nphases array of phase guesses

*:param combine_bands_mode:* "fairSIM" if using method of https://doi.org/10.1038/ncomms10980 or "openSIM" if
using method of https://doi.org/10.1109/jstqe.2016.2521542\ :param float fmax_exclude_band0: amount of the unshifted bands to exclude, as a fraction of fmax. This can enhance optical sectioning by replacing the low frequency information in the reconstruction with the data. from the shifted bands only. For more details on the band replacement optical sectioning approach, see https://doi.org/10.1364/BOE.5.002580\ and https://doi.org/10.1016/j.ymeth.2015.03.020\ 

*:param mod_depths_guess:* If use_fixed_mod_depths is True, these modulation depths are used

*:param bool use_fixed_mod_depths:* if true, use mod_depths_guess instead of estimating the modulation depths from the data

*:param bool normalize_histograms:* for each phase, normalize histograms of images to account for laser power fluctuations

*:param background:* Either a single number, or broadcastable to size of imgs. The background will be subtracted
 before running the SIM reconstruction

*:param bool determine_amplitudes:* whether or not to determine amplitudes as part of Wicker phase optimization. This flag only has an effect if phase_estimation_mode is "wicker-iterative"

*:param gain:* gain of the camera in ADU/photons. This is a single number or an array which is broadcastable to the same size as the images whcih is used to convert the ADU counts to photon numbers.

*:param max_phase_err:* If the determined phase error between components exceeds this value, use the phase guess values instead of those determined by the estimation algorithm.

*:param min_p2nr:* if the peak-to-noise ratio is smaller than this value, use the frequency guesses instead of the frequencies determined by the estimation algorithm.

*:param fbounds:* frequency bounds as a fraction of fmax to be used in power spectrum fit. todo: remove ...

*:param bool interactive_plotting:* show plots in python GUI windows, or save outputs only

*:param str save_dir:* directory to save results. If None, then results will not be saved

*:param bool use_gpu:* use GPU for processing if available

*:param figsize:* figure size for diagnostic plots

### Perform SIM reconstruction using known OTF and plot results. Loop over multiple Wiener parameter values to see effect on reconstruction.

In [ ]:
%matplotlib inline

# path to save data
save_path = Path("data","example_002","reconstruction")

# define list of wiener parameters to test in reconstruction
w_idx = 0
wiener_parameters = [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4]

# loop over all wiener parameters
for wiener_parameter in wiener_parameters:

    # create mcSIM reconstruction object. See docstring for details on each parameter.
    imgset = sim.SimImageSet({"pixel_size": pixel_size, "na": na, "wavelength": emission_wavelengths[0]},
                            imgs[0, :, :, roi[0]:roi[1], roi[2]:roi[3]],
                            frq_estimation_mode="band-correlation",
                            frq_guess=frqs_guess,
                            phases_guess=phases_guess,
                            phase_estimation_mode="wicker-iterative",
                            combine_bands_mode="fairSIM",
                            fmax_exclude_band0=0.4, 
                            normalize_histograms=True,
                            otf=otf,
                            wiener_parameter=wiener_parameter,
                            background=100, 
                            gain=2, 
                            min_p2nr=0.5,
                            save_dir=save_path,
                            save_suffix="_w"+str(w_idx).zfill(3),
                            interactive_plotting=False, 
                            figsize=(20, 13))

    # perform reconstruction, plot figures, save, and clean up log file
    imgset.reconstruct()
    imgset.plot_figs()
    imgset.save_imgs()
    imgset.save_result()

    # create variables to hold widefield and SIM SR images
    if w_idx == 0:
        wf_images = np.zeros((len(wiener_parameters),imgset.widefield.shape[0],imgset.widefield.shape[1]),dtype=np.float32)
        SR_images = np.zeros((len(wiener_parameters),imgset.sim_sr.shape[0],imgset.sim_sr.shape[1]),dtype=np.float32)

    # store widefield and SIM SR images for display
    wf_images[w_idx,:]= imgset.widefield
    SR_images[w_idx,:] = imgset.sim_sr
    w_idx = w_idx + 1

    # clean up mcSIM reconstruction object
    del imgset

### Display results

In [ ]:
# define colormaps
colormaps = ['bop blue', 'bop orange']

# add images to napari viewer with scale information, colormaps, and additive blending
viewer = napari.view_image(wf_images,name='Widefield',scale=(pixel_size,pixel_size),colormap = colormaps[0],blending='additive',contrast_limits=[0,2**16-1])
viewer.add_image(SR_images,name='SR',scale=(pixel_size/2,pixel_size/2),colormap = colormaps[1],blending='additive',contrast_limits=[0,2**16-1])

# activate scale bar in physical units
viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True

# label sliders in napari viewer
viewer.dims.axis_labels = ['w_idx','y','x']

## Importance of SLM calibration

### Calculate FFT of raw data to find frequency guesses

In [ ]:
%matplotlib widget

# loop over all channels
for ch_idx in range(ncolors):

    # extract angle/phase data for current channel
    image_set = imgs[ch_idx,:,:,:,:]

    # create grid containing correct spatial frequencies given image sampling
    dx = pixel_size
    fxs = analysis_tools.get_fft_frqs(nx, dx)
    df = fxs[1] - fxs[0]
    fys = analysis_tools.get_fft_frqs(ny, dx)
    ff = np.sqrt(np.expand_dims(fxs, axis=0)**2 + np.expand_dims(fys, axis=1)**2)

    # loop over all angle/phases and plot absolute value of 2D fourier transform
    for ii in range(image_set.shape[0]):
        ft = fft.fftshift(fft.fft2(fft.ifftshift(np.squeeze(image_set[ii, 0, :]))))

        figh = plt.figure()
        plt.title('Z='+str(ch_idx)+", Angle="+str(ii))
        plt.imshow(np.abs(ft), norm=PowerNorm(gamma=0.1,vmin=65,vmax=1.5e7),
                    extent=[fxs[0] - 0.5 * df, fxs[-1] + 0.5 * df, fys[-1] + 0.5 * df, fys[0] - 0.5 * df])
        plt.show()